In [11]:
#Include all packages
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
import pandas as pd


In [13]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
if len(physical_devices) > 0:  # If you have at least one "configured" GPU, let's use it; otherwise, pass
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Using GPU")


[]


In [14]:
def LoadDataSet(dataSetFolderPath: str):
    images = []
    croppedValues = []
    annotationsFilePath = dataSetFolderPath+"/allAnnotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath,sep=";")
    width, height = 500, 500
    for index, row in annotationsDataFrame[1:].iterrows():
        image = cv2.imread(dataSetFolderPath+"/"+row[0])
        # cv2.imshow("image", image)
        resized_img  = cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)
        # cv2.imshow("resized_img", resized_img)
        images.append(resized_img)
        croppedValues.append((row[2],row[3],row[4],row[5]))

    del annotationsDataFrame
    return images, croppedValues



In [15]:
annotationsFilePath = "./DataSet"+"/allAnnotations.csv"
annotationsDataFrame = pd.read_csv(annotationsFilePath,sep=";")

In [17]:
images, croppedValues = LoadDataSet("./DataSet")
X_train, X_val = train_test_split(images, test_size=0.3, random_state=42)
del images